In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

SAVEFIG = True

sns.set_style('darkgrid')
plt.rcParams.update({'font.size': 9,
                     'ytick.labelsize': 9,
                     'xtick.labelsize': 9})

In [2]:
FILENAME = 'tcm5_dataset_5'
df = pd.read_csv(rf'./data_zenodo/{FILENAME}.csv')
df.columns = [ i.replace("work_roll", "wr") for i in df.columns]

anomaly_columns = [c for c in df.columns if 'Anomaly' in c]
feature_columns = [c for c in df.columns if c not in anomaly_columns]
X = df.loc[:, feature_columns]
Y = df.loc[:, anomaly_columns]
y = Y.sum(axis=1) != 0

In [ ]:
X_normal = X.loc[y == False]
X_anomaly = X.loc[y == True]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_normal)

X_scaled = scaler.transform(X)

fig = plt.figure(figsize=(12, 3))
ax = plt.subplot()
sns.boxplot(X_scaled, fliersize=1, width=0.6, zorder=4)
ax.axhline(0.0, color='black', linestyle='--', linewidth=1)
ax.axhline(1.0, color='black', linestyle='--', linewidth=1)
ax.set_xticks([i for i in range(len(X.columns))], [i for i in X.columns], rotation=90)
ax.set_yticks([0, 1], ['Normal MIN', 'Normal MAX'])
ax.set_xlim(-0.5, len(X.columns)-0.5)

fig.tight_layout()
if SAVEFIG:
    fig.savefig(f'figures_zenodo/{FILENAME}_box_summary.pdf', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = plt.subplot()
X_corr = X.corr()
sns.heatmap(X_corr, zorder=4, ax=ax, cmap='viridis', vmin=-1, vmax=1.0, cbar_kws={'label': 'correlation coefficient', 'location': 'top', 'aspect': 50})
fig.tight_layout()

if SAVEFIG:
    fig.savefig(f'figures_zenodo/{FILENAME}_corr.pdf', bbox_inches='tight')

In [ ]:
i = 14200
n = 600
X_plot = X.iloc[i:i+n]
y_plot = y.iloc[i:i+n]

feature = 'reduction_1'
anomaly_column = 'Anomaly_Reduction'

X_anomaly = X_plot.loc[Y[anomaly_column]]

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 3))
sc = axes.scatter(X_plot.index, X_plot[feature], s=8, c=X_plot['thickness_entry'], cmap='viridis')
plt.colorbar(sc, label='entry thickness [mm]', pad=0.01)
axes.scatter(X_anomaly.index, X_anomaly[feature], color='None', edgecolor='red', s=100, linewidth=1.5, label=anomaly_column, alpha=0.7)
axes.set_xlim(X_plot.index[0], X_plot.index[-1])
axes.set_ylabel(feature)
axes.set_xlabel("No. observation")
fig.legend(loc=1, bbox_to_anchor=(0.83, 1.03), fontsize=8, alignment='left')
fig.tight_layout(rect=[0.01, 0.01, 0.99, 0.99])

if SAVEFIG:
    fig.savefig(f'figures_zenodo/{FILENAME}_stream_{anomaly_column}.pdf', bbox_inches='tight')


In [ ]:
feature_types = [
    'force',
    'reduction',
    'roll_speed',
    'motor_power',  
    'gap',
    'torque'
    ]

ncols = 3
nrows = int(len(feature_types) // ncols + np.ceil(len(feature_types)%ncols / len(feature_types) ))
figsize = (3.7 * ncols, 2 * nrows)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)

legend = True
for feature_type, ax in zip(feature_types, axes.flat):
    stand_features = [f for f in X.columns if feature_type in f]
    
    for sf in stand_features:
        stand_no = sf[-1]
        if legend:
            label=f"S{stand_no}"
        else:
            label = None

        sns.kdeplot(data=X, x=sf, label=label, ax=ax, fill=True, alpha=0.2)
    legend = False
    ax.set_xlabel(feature_type)
    ax.set_yticks([])
    ax.set_ylabel(None)

for i in range(nrows):
    ax = axes[i][0]
    ax.set_ylabel("Probability")

fig.legend(fontsize=9, ncols=6, bbox_to_anchor=(0.015, 1.08), loc=2, frameon=False, title='Stand')
fig.tight_layout()
if SAVEFIG:
    fig.savefig(f'figures_zenodo/{FILENAME}_hist_features.pdf', bbox_inches='tight')

In [ ]:
feature_types = [
    'roll_speed',
    'reduction',
    'force',
    'torque',
    'motor_power',
]

for si in range(1, 6):

    features = [col for col in X.columns if f"_{si}" in col]
    features = [f for f in features if f[:-2] in feature_types]
    ncols = 5
    nrows = int(len(features) // ncols + np.ceil(len(features)%ncols / len(features) ))
    figsize = (2* ncols, 2 * nrows)

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    X_normal = X.loc[y == 0]
    X_anomaly = X.loc[y != 0]

    legend = True
    for col, ax in zip(features, axes.flat):
        for Xi, label in zip([X_normal, X_anomaly], ['Normal', 'Anomaly']):
            if legend:
                label2=label
            else:
                label2=None
            sns.histplot(Xi[col], label=None, alpha=0.2, bins=15, ax=ax, stat='density', kde=False)
            sns.kdeplot(Xi[col], label=label2, ax=ax, linewidth=2)
        if legend:
            
            legend = False
        ax.set_xlabel(col)
        ax.set_yticks([])
        ax.grid(False)

    axes.flat[0].set_ylabel('Probability')
    for ax in axes.flat[1:]:
        ax.set_ylabel('')
    
    fig.legend(fontsize=10, ncols=2, bbox_to_anchor=(0.015, 1.1), loc=2, frameon=False)
    fig.tight_layout()
    
    if SAVEFIG:
        fig.savefig(f'figures_zenodo/{FILENAME}_histograms_{si}.pdf', bbox_inches='tight')

In [ ]:
window = 500
X_normal = X.loc[y == 0]
X_anomaly = X.loc[y > 0]
X_normal_rolling = X_normal.rolling(window=window).mean()

# create samples
X_normal = X_normal.sample(frac=0.1, random_state=42).sort_index()
X_anomaly = X_anomaly.sample(frac=0.3, random_state=42).sort_index()

drift_columns = ['roll_speed_5', 'motor_power_5']

nrows = 1 + len(drift_columns)
fig, axes = plt.subplots(nrows=nrows, ncols=1, figsize=(10, 2*nrows))

label_normal = 'Normal'
label_anomaly = 'Anomaly'
label_ma = 'Moving Average'

for i, col in enumerate(drift_columns):
    ax = axes[i]
    ax.set_ylabel(col)
    ax.scatter(X_normal.index, X_normal[col], s=8, alpha=0.3, label=label_normal)
    ax.scatter(X_anomaly.index, X_anomaly[col], s=8, alpha=1.0, label=label_anomaly)
    ax.plot(X_normal_rolling.index, X_normal_rolling[col], color='tab:blue', label=label_ma, linewidth=2)
    ax.set_xlim(window, len(X))
    
    label_normal = None
    label_anomaly = None
    label_ma = None


# products
def rolling_unique_count(window):
    return len(set(window))
product_label = (X['thickness_entry'] * X['thickness_exit'] * X['ys_entry'])

unique_counts = product_label.rolling(window=window).apply(lambda x: rolling_unique_count(x), raw=False)
ax = axes[-1]
ax.set_ylabel(f'No. unique products\n(prev. {window} coils)')
ax.set_xlabel('No. observations')
ax.plot(X.index, unique_counts, color='tab:blue', label=None, linewidth=2)
ax.set_xlim(window, len(X))
fig.tight_layout()

fig.legend(loc=2, ncol=3, bbox_to_anchor=(0.05, 1.05))
fig.tight_layout()

if SAVEFIG:
    fig.savefig(f'figures_zenodo/{FILENAME}_drift.pdf', bbox_inches='tight')


### Anomalies

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 6))


def plot_anomaly_scatter(X, T, anomaly_col, x_col, y_col, ax=None):
    if ax is None:
        ax = plt.gca()

    X_normal = X.loc[Y[anomaly_col] == 0]
    X_anomaly = X.loc[Y[anomaly_col] > 0]
    X_all_anomaly = X.loc[Y.sum(axis=1) != 0]
    label = anomaly_col.replace("_", " ").replace("WorkRoll", "Work Roll")
    ax.set_title(label)
    
    color_feature = 'thickness_exit'
    vmin = X[color_feature].min()
    vmax = X[color_feature].max()
    mappable = ax.scatter(X[x_col], X[y_col], c=X[color_feature], vmin=vmin, vmax=vmax, cmap='viridis', s=3)
    ax.scatter(X_anomaly[x_col], X_anomaly[y_col], c=X_anomaly[color_feature], vmin=vmin, vmax=vmax, cmap='viridis', s=3)
    ax.scatter(X_all_anomaly[x_col], X_all_anomaly[y_col], s=10, label='Other Anomalies', edgecolor='tab:orange', color='None', alpha=0.3)
    ax.scatter(X_anomaly[x_col], X_anomaly[y_col], s=40, label=label, edgecolor='tab:red', color='None')
    ax.set(xlabel=x_col, ylabel=y_col)
    ax.legend()

    return mappable

sc = plot_anomaly_scatter(X, Y, 'Anomaly_Reduction', 'reduction_1', 'reduction_2', ax=axes.flat[0])
sc = plot_anomaly_scatter(X, Y, 'Anomaly_WorkRoll_1', 'reduction_1', 'force_1', ax=axes.flat[1])
sc = plot_anomaly_scatter(X, Y, 'Anomaly_Bearing_1', 'force_1', 'torque_1', ax=axes.flat[2])
sc = plot_anomaly_scatter(X, Y, 'Anomaly_Electric_1', 'torque_1', 'motor_power_1', ax=axes.flat[3])

fig.tight_layout(rect=[0.02, 0.05, 0.98, 0.98])
cbar_ax = fig.add_axes([0.15, 0.03, 0.7, 0.02])
cbar = fig.colorbar(sc, cax=cbar_ax, orientation='horizontal', label='thickness_exit')

if SAVEFIG:
    fig.savefig(f'figures_zenodo/{FILENAME}_anomalies.pdf', bbox_inches='tight')

### PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler
pca = PCA(n_components=2)
scaler = StandardScaler()
scaler.fit(X_normal)
X_scaled = scaler.transform(X)
X_pca = pca.fit_transform(X_scaled)
y = Y.sum(axis=1) != 0
X_pca_normal = X_pca[y == 0][::3]
X_pca_anomaly = X_pca[y != 0]

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4.5, 4))
ax.scatter(X_pca_normal[:, 0], X_pca_normal[:, 1], label='Normal', s=6)
ax.scatter(X_pca_anomaly[:, 0], X_pca_anomaly[:, 1], label='Anomaly', s=3, edgecolor='tab:orange', color='tab:orange', alpha=0.7)
    
ax.set(xlabel='PC1', ylabel='PC2')
ax.legend(ncol=2, loc=2)
fig.tight_layout()
if SAVEFIG:
    fig.savefig(f'figures/{FILENAME}_pca.pdf', bbox_inches='tight')

### summary

In [ ]:
import os
import pandas as pd
dirname = r'./data_zenodo'
files = os.listdir(dirname)

def get_anomaly_types(df):
    anomaly_types = df[[i for i in df.columns if 'Anomaly' in i]].any(axis=0)
    anomaly_types = anomaly_types.loc[anomaly_types==True].index
    return len(anomaly_types)


def get_total_anomaly_types(df):
    anomaly_types = df[[i for i in df.columns if 'Anomaly' in i]].any(axis=0)
    anomaly_types = anomaly_types.loc[anomaly_types==True].index
    anomaly_types = [" ".join(x.split("_")[:-1]) for x in anomaly_types]
    anomaly_types = set(anomaly_types)
    return len(anomaly_types)

def get_products(df):
    dfc = df.copy(deep=True)
    def extract_product(x):
        ys0 = x['ys_entry']
        ithick = x['thickness_entry']
        othick = x['thickness_exit']
        width = x['width']
        return  f"{ys0:.0f}-{ithick}-{othick}-{width:.0f}"
    dfc['product'] = df.apply(lambda x: extract_product(x), axis=1)
    n = len(dfc['product'].unique())
    return n

table = pd.DataFrame()
for i, f in enumerate(files):
    
    path = os.path.join(dirname, f)
    df = pd.read_csv(path)
    table.at[i, 'Dataset'] = f[:-4]
    table.at[i, 'Observations'] = len(df)
    
    table.at[i, 'Anomalies'] = df[[i for i in df.columns if 'Anomaly' in i]].any(axis=1).sum()
    table.at[i, 'Share of Anomalies'] = f"{table.at[i, 'Anomalies'] / table.at[i, 'Observations']*100:.1f}%"
    table.at[i, "Features"] = df[[i for i in df.columns if 'Anomaly' not in i]].shape[-1]
    total_anomaly_types = get_total_anomaly_types(df)
    all_anomaly_types = get_anomaly_types(df)
    table.at[i, "Anomaly Types"] = f"{total_anomaly_types} ({all_anomaly_types})"
    table.at[i, "Products"] = get_products(df)
    table.at[i, "Data Drift"] = int(f[-5]) >= 5


for col in ['Observations', 'Anomalies', 'Features', "Anomaly Types", "Products"]:
    if table[col].dtype != 'object':
        table[col] = pd.to_numeric(table[col], downcast='integer')

table

In [ ]:
cols = list(df.columns)
for i, c in enumerate(cols):
    if c[-2] == "_":
        cols[i] =c[:-2]

cols_unique = []
for col in cols:
    if col not in cols_unique:
        cols_unique.append(col)

feature_table = pd.DataFrame()
for i, feature in enumerate(cols_unique):
    feature_table.at[i, "Feature"] = feature
    fcols = [c for c in df.columns if feature in c]
    if len(fcols) == 1:
        feature_table.at[i, "Suffixes"] = "-"
    else:
        smin = fcols[0][-1]
        smax = fcols[-1][-1]
        feature_table.at[i, "Suffixes"] = f"{smin} to {smax}"

    if feature in ['thickness_entry', 'thickness_exit', 'width', 'work_roll_diam', 'gap']:
        feature_table.at[i, "Unit"] = "mm"
    elif 'mileage' in feature:
        feature_table.at[i, "Unit"] = "km"
    elif feature in ['force', 'tension']:
        feature_table.at[i, "Unit"] = "N"

    elif feature == 'torque':
        feature_table.at[i, "Unit"] = "Nm"
    elif "ys_" in feature:
        feature_table.at[i, "Unit"] = "MPa"
    elif feature == 'motor_power':
        feature_table.at[i, "Unit"] = "kW"
    elif 'Anomaly' in feature or feature =='reduction':
        feature_table.at[i, "Unit"] = "-"
    feature_table.at[i, "Description"] = ""
feature_table